# core

> SFT LLM

In [ ]:
#| default_exp core
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

# Phi2 SFT training baseline
data
use all public data, but I dropped the dupilcate rewrite prompts.

hyperparamters
epoch: 5

batch size: 2

gradient_accumulation_steps: 8

max_seq_length: 1024

learing rate: 1e-4

inference notebook click here

In [ ]:
def train_phi():
    exp_name = 'phi2_public_data_sft'
    data_path = '/kaggle/input/llmpr-public-10k-unique/public_10k_unique_rewrite_prompt.csv'
    model_path = '/kaggle/input/phi/transformers/2/1'
    output_path = f'outputs'
    model_save_path =  f'{exp_name}_adapter'

    epochs=5
    batch_size=1 # 2 
    max_seq_length=512 # 1024 
    lr = 1e-4

    tokenizer = AutoTokenizer.from_pretrained(
        model_path,
        )
    tokenizer.pad_token = tokenizer.eos_token


    bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type='nf4',
            bnb_4bit_compute_dtype='float16',
            bnb_4bit_use_double_quant=False,
        )

    model = AutoModelForCausalLM.from_pretrained(model_path,
                                                 quantization_config=bnb_config,
                                                 trust_remote_code=True,
                                                 use_auth_token=True)

    model.config.gradient_checkpointing = False

    def token_len(text):
        tokenized = tokenizer(text, return_length=True)
        length = tokenized['length'][0]
        return length
        
    def formatting_prompts_func(example):
        output_texts = []
        for i in range(len(example['rewritten_text'])):
            ori_text = example['original_text'][i]
            rew_text = example['rewritten_text'][i]
            rew_prompt = example['rewrite_prompt'][i]
            text = f"Instruct: Original Text:{ori_text}\nRewritten Text:{rew_text}\nWrite a prompt that was likely given to the LLM to rewrite original text into rewritten text.Output: {rew_prompt}"
            if token_len(text) > max_seq_length:
                continue
            output_texts.append(text)
        return output_texts

    response_template = "Output:"
    collator = DataCollatorForCompletionOnlyLM(response_template=response_template, 
                                               tokenizer=tokenizer)

    peft_config = LoraConfig(
        r=16,
        lora_alpha=32,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ["q_proj", "k_proj", "v_proj", "dense"],
    )
    
    args = TrainingArguments(
        output_dir = output_path,
        fp16=True,
        learning_rate=lr,
        optim="adafactor",
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size*2,
        gradient_accumulation_steps=8,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=1,
        logging_steps=50,
        lr_scheduler_type="cosine",
        warmup_ratio=0.1,
        weight_decay=0.01,
        report_to='none',
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        )
    
    
    trainer = SFTTrainer(
        model=model,
        args = args,
        max_seq_length=max_seq_length,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        formatting_func=formatting_prompts_func,
        data_collator=collator,
        peft_config=peft_config,
    )
    
    
    trainer.train()
    
    trainer.save_model(model_save_path)
    tokenizer.save_pretrained(model_save_path)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()